In [1]:
import pickle
import pathlib

path_data = pathlib.Path().cwd().parent / 'data'

with open(path_data / 'X.pickle', 'rb') as f:
    X = pickle.load(f)

with open(path_data / 'y.pickle', 'rb') as f:
    y = pickle.load(f)

In [2]:
import pickle
import pathlib
import optuna
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

/home/esdrasgc/Insper/8_semestre/redes-neurais/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configurações
N_TRIALS = 20  # Número de testes - pode ser ajustado aqui
RANDOM_STATE = 42
CV_FOLDS = 4

In [4]:
def objective(trial):
    """Função objetivo para otimização com Optuna"""
    
    # Hiperparâmetros para otimizar
    hidden_layer_sizes = []
    n_layers = trial.suggest_int('n_layers', 1, 5)
    
    for i in range(n_layers):
        layer_size = trial.suggest_int(f'layer_{i}_size', 50, 500)
        hidden_layer_sizes.append(layer_size)
    
    hidden_layer_sizes = tuple(hidden_layer_sizes)
    
    alpha = trial.suggest_float('alpha', 1e-6, 1e-1, log=True)
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-5, 1e-1, log=True)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    solver = trial.suggest_categorical('solver', ['adam', 'lbfgs'])
    
    # Criar pipeline com normalização e modelo
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,
            alpha=alpha,
            learning_rate_init=learning_rate_init,
            activation=activation,
            solver=solver,
            max_iter=1000,
            random_state=RANDOM_STATE,
            early_stopping=True,
            validation_fraction=0.1
        ))
    ])
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
    
    return scores.mean()

# Criar estudo Optuna
study = optuna.create_study(direction='maximize')

print(f"Iniciando otimização com {N_TRIALS} trials...")
study.optimize(objective, n_trials=N_TRIALS)

print("Otimização concluída!")
print(f"Melhor score: {study.best_value:.4f}")
print(f"Melhores parâmetros: {study.best_params}")

[I 2025-09-15 13:07:29,503] A new study created in memory with name: no-name-232f4fdb-0995-4530-a21f-ddbf95b91090


Iniciando otimização com 20 trials...


[I 2025-09-15 13:08:21,220] Trial 0 finished with value: 0.56925 and parameters: {'n_layers': 3, 'layer_0_size': 205, 'layer_1_size': 125, 'layer_2_size': 389, 'alpha': 0.03886907184228022, 'learning_rate_init': 1.3895969860385052e-05, 'activation': 'relu', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.56925.
[I 2025-09-15 13:11:10,703] Trial 1 finished with value: 0.567875 and parameters: {'n_layers': 5, 'layer_0_size': 145, 'layer_1_size': 267, 'layer_2_size': 400, 'layer_3_size': 332, 'layer_4_size': 175, 'alpha': 0.00024628101566735613, 'learning_rate_init': 0.00021324073159674893, 'activation': 'relu', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.56925.
[I 2025-09-15 13:11:15,625] Trial 2 finished with value: 0.6868750000000001 and parameters: {'n_layers': 1, 'layer_0_size': 451, 'alpha': 0.012020023353362531, 'learning_rate_init': 0.0017362016761521576, 'activation': 'logistic', 'solver': 'adam'}. Best is trial 2 with value: 0.6868750000000001.
[I 2025-09-15 13:11:27,198

Otimização concluída!
Melhor score: 0.6869
Melhores parâmetros: {'n_layers': 1, 'layer_0_size': 451, 'alpha': 0.012020023353362531, 'learning_rate_init': 0.0017362016761521576, 'activation': 'logistic', 'solver': 'adam'}
